# Sélection des caractéristiques

## Valeurs manquantes

+ Supprimer les colonnes suivantes du DataFrame numerical_train (que je vous ai noté):
 - PID (l'identifiant de la maison n'est pas utile pour la modélisation)
 - Year Built
 - Year Remod/Add
 - Garage Yr Blt
 - Mo Sold
 - Yr Sold
+ Calculer le nombre de valeurs manquantes de chaque colonne de numerical_train. Créer un objet Series dont l'index est composé de noms de colonnes et dont les valeurs associées correspondent au nombre de valeurs manquantes.
+ Assigner l'objet Series à la variable null_series. Sélectionner le de null_series pour garder seulement les colonnes avec aucune valeur manquante et assigner l'objet Series résultant à la variable full_cols_series.
+ Afficher full_cols_series.

In [1]:
import pandas as pd
data = pd.read_csv('AmesHousing.txt', delimiter="\t")
train = data[0:1460]
test = data[1460:]

# pandas.DataFrame.drop()
# pandas.DataFrame.isnull()
# pandas.DataFrame.sum()

# df.isnull().sum()

numerical_train = train.select_dtypes(include=['int', 'float'])
numerical_train = numerical_train.drop(['PID', 'Year Built', 'Year Remod/Add', 'Garage Yr Blt',
                                       'Mo Sold', 'Yr Sold'], axis=1)
null_series = numerical_train.isnull().sum()
full_cols_series = null_series[null_series == 0]
print(full_cols_series)

Order              0
MS SubClass        0
Lot Area           0
Overall Qual       0
Overall Cond       0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
TotRms AbvGrd      0
Fireplaces         0
Garage Cars        0
Garage Area        0
Wood Deck SF       0
Open Porch SF      0
Enclosed Porch     0
3Ssn Porch         0
Screen Porch       0
Pool Area          0
Misc Val           0
SalePrice          0
dtype: int64


## Corrélation des colonnes de caractéristiques avec la colonne cible

+ Calculer les coefficients de corrélation par paire entre toutes les colonnes de train_subset (DataFrame filtré sur les index de l'objet series full_colls_series contenant des colonnes sans aucune valeur manquante).
+ Sélectionner uniquement la colonne SalePrice dans le DataFrame résultant. 
+ Calculer la valeur absolue de chaque terme, trier l'objet Series résultant en fonction des valeurs de corrélation et assigner le résultat à la variable sorted_corrs.
+ Afficher le résultat.

In [2]:
# pandas.Series.abs()

# pandas.Series.sort_values()

In [3]:
train_subset = train[full_cols_series.index]

corrmat = train_subset.corr()
sorted_corrs = corrmat['SalePrice'].abs().sort_values()
print(sorted_corrs)

Misc Val           0.009903
3Ssn Porch         0.038699
Low Qual Fin SF    0.060352
Order              0.068181
MS SubClass        0.088504
Overall Cond       0.099395
Screen Porch       0.100121
Bedroom AbvGr      0.106941
Kitchen AbvGr      0.130843
Pool Area          0.145474
Enclosed Porch     0.165873
2nd Flr SF         0.202352
Half Bath          0.272870
Lot Area           0.274730
Wood Deck SF       0.319104
Open Porch SF      0.344383
TotRms AbvGrd      0.483701
Fireplaces         0.485683
Full Bath          0.518194
1st Flr SF         0.657119
Garage Area        0.662397
Garage Cars        0.663485
Gr Liv Area        0.698990
Overall Qual       0.804562
SalePrice          1.000000
Name: SalePrice, dtype: float64


## Heatmap de la matrice de corrélation

+ Sélectionner seulement les colonnes de sorted_corrs dont la corrélation est supérieure 0.3 et assigner le résultat à la variable strong_corrs.
+ Utiliser la fonction seaborn.heatmap() pour générer une heatmap de matrice de corrélation pour les colonnes de strong_corrs.

In [4]:
# .index

In [5]:
import seaborn as sns

strong_corrs = sorted_corrs[sorted_corrs > 0.3]
corrmat = train_subset[strong_corrs.index].corr()
sns.heatmap(corrmat)

## Entrainer et tester le modèle

+ Filtrer le dataframe test afin qu'il ne contienne que les colonnes de final_corr_cols.index. Ensuite, supprimer la ligne contenant la valeur manquante et assigner le résultat à la variable clean_test
+ Construire un modèle de régression linéaire en utilisant les caractéristiques de features (c’est-à-dire les colonnes sélectionnées excepté la colonne cible) et la colonne cible SalePrice.
+ Calculer la valeur de RMSE sur les sets de training et de test.
+ Assigner la valeur RMSE pour l'entrainement à la variable train_rmse et celle du test à la variable test_rmse.

In [6]:
# DataFrame.dropna()

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

final_corr_cols = strong_corrs.drop(['Garage Cars', 'TotRms AbvGrd'])
features = final_corr_cols.drop(['SalePrice']).index
target = 'SalePrice'

clean_test = test[final_corr_cols.index].dropna()

lr = LinearRegression()
lr.fit(train[features], train['SalePrice'])

train_predictions = lr.predict(train[features])
test_predictions = lr.predict(clean_test[features])

train_mse = mean_squared_error(train_predictions, train[target])
test_mse = mean_squared_error(test_predictions, clean_test[target])

train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

print(train_rmse)
print(test_rmse)

34173.97629185852
41032.0261201977


## Supprimer les caractéristiques de faible variance

+ Sélectionner les colonnes de features dans le DataFrame train. Redimensionner chaque colonne pour que les valeurs oscillent entre 0 et 1.
+ Calculer et afficher les valeurs min et max de la colonne pour vous assurer qu'elles sont comprises entre 0 et 1.
+ Calculer la variance de ces colonnes, trier l'objet Series résultant en fonction de ses valeurs et assigner le résultat à la variable sorted_vars.
+ Afficher le resultat sorted_vars.

In [8]:
# DataFrame.max()

In [9]:
unit_train = (train[features] - train[features].min())/(train[features].max() - train[features].min())
print(unit_train.min())
print(unit_train.max())

sorted_vars = unit_train.var().sort_values()
print(sorted_vars)

Wood Deck SF     0.0
Open Porch SF    0.0
Fireplaces       0.0
Full Bath        0.0
1st Flr SF       0.0
Garage Area      0.0
Gr Liv Area      0.0
Overall Qual     0.0
dtype: float64
Wood Deck SF     1.0
Open Porch SF    1.0
Fireplaces       1.0
Full Bath        1.0
1st Flr SF       1.0
Garage Area      1.0
Gr Liv Area      1.0
Overall Qual     1.0
dtype: float64
Open Porch SF    0.013938
Full Bath        0.018621
Garage Area      0.020347
Gr Liv Area      0.023078
Overall Qual     0.024496
1st Flr SF       0.025814
Wood Deck SF     0.033064
Fireplaces       0.046589
dtype: float64


## Modèle final

+ Construire un modèle de régression linéaire avec les caractéristiques restantes.
+ Calculer la valeur RMSE sur les sets de train et de test.
+ Assigner la valeur RMSE de l'entrainement à la variable train_rmse_2 et la valeur RMSE de test à la variable test_rmse_2.
+ Afficher les 2 valeurs.

In [10]:
clean_test = test[final_corr_cols.index].dropna()
features = features.drop('Open Porch SF')

lr = LinearRegression()
lr.fit(train[features], train['SalePrice'])

train_predictions = lr.predict(train[features])
test_predictions = lr.predict(clean_test[features])

train_mse = mean_squared_error(train_predictions, train['SalePrice'])
test_mse = mean_squared_error(test_predictions, clean_test['SalePrice'])

train_rmse_2 = np.sqrt(train_mse)
test_rmse_2 = np.sqrt(test_mse)

print(train_rmse_2)
print(test_rmse_2)

34372.696707783965
40591.42702437726
